In [1]:
import requests
from bs4 import BeautifulSoup
import json
import os
import html
from time import sleep

RESULTS_DIR = "parsing_results_DimaP"


def parse_description_fallback(url: str):
    """Пробуем вытащить описание с разными селекторами"""
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    description = ""
    whats_the_point = ""

    # 1. Старый вариант (описание в <p>)
    desc_block = soup.select_one(
        "main div:nth-of-type(5) div div div div div:nth-of-type(2)"
    )
    if desc_block:
        ps = desc_block.select("p")
        if ps:
            description = html.unescape(ps[0].get_text(" ", strip=True))
        if len(ps) > 1:
            whats_the_point = html.unescape(ps[1].get_text(" ", strip=True))

    # 2. Новый вариант (описание в <div class="HtmlContent">)
    if not description:
        html_block = soup.select_one("div.HtmlContent")
        if html_block:
            description = html.unescape(html_block.get_text(" ", strip=True))

    # 3. Fallback — ищем большой кусок текста внутри main
    if not description:
        candidates = [
            div.get_text(" ", strip=True)
            for div in soup.select("main div")
            if len(div.get_text(strip=True)) > 100
        ]
        if candidates:
            description = candidates[0]

    return description, whats_the_point


def refill_missing_descriptions(
    input_file="shows.json", output_file="shows_fixed.json"
):
    path = os.path.join(RESULTS_DIR, input_file)
    with open(path, "r", encoding="utf-8") as f:
        shows = json.load(f)

    updated = 0
    for show in shows:
        if not show.get("description"):
            print(f"🔄 Допарсим: {show['title']}")
            try:
                desc, wtp = parse_description_fallback(show["url"])
                if desc:
                    show["description"] = desc
                if wtp and not show.get("whats_the_point"):
                    show["whats_the_point"] = wtp
                updated += 1
                sleep(0.2)  # задержка, чтобы не долбить сайт
            except Exception as e:
                print(f"⚠️ Ошибка у {show['title']}: {e}")

    # сохраняем результат
    output_path = os.path.join(RESULTS_DIR, output_file)
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(shows, f, ensure_ascii=False, indent=2)

    print(f"✅ Дособрано {updated} описаний. Результат сохранён в {output_path}")


if __name__ == "__main__":
    refill_missing_descriptions()

🔄 Допарсим: Игра в кальмара
🔄 Допарсим: Бумажный дом
🔄 Допарсим: Грань
🔄 Допарсим: Дом дракона
🔄 Допарсим: Разделение
🔄 Допарсим: Фоллаут
🔄 Допарсим: Универ. Новая общага
🔄 Допарсим: Ну, погоди!
🔄 Допарсим: Истребитель демонов
🔄 Допарсим: Фишер
🔄 Допарсим: Песочный человек
🔄 Допарсим: В её глазах
🔄 Допарсим: Чёрная птица
🔄 Допарсим: Приключения Шерлока Холмса и доктора Ватсона
🔄 Допарсим: Нулевой пациент
🔄 Допарсим: Человек-бензопила
🔄 Допарсим: Изобретая Анну
🔄 Допарсим: 1899
🔄 Допарсим: Поколение «Ви»
🔄 Допарсим: Каштановый человечек
🔄 Допарсим: Переполненная комната
🔄 Допарсим: Падение дома Ашеров
🔄 Допарсим: Пингвин
🔄 Допарсим: Грешники
🔄 Допарсим: Счастливы вместе
🔄 Допарсим: Женщина-Халк: Адвокат
🔄 Допарсим: Терапия
🔄 Допарсим: Аутсорс
🔄 Допарсим: Пацанки
🔄 Допарсим: Монолог фармацевта
🔄 Допарсим: This is Хорошо
🔄 Допарсим: Этим летом я стала красивой
🔄 Допарсим: Джек Ричер
🔄 Допарсим: Леди Баг и Супер-кот
🔄 Допарсим: Уборщица. История матери-одиночки
🔄 Допарсим: Медленные лошади